In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
# Cargar y parsear el XML manualmente
tree = ET.parse("/mnt/c/Users/Administrador/Downloads/ReportOptimizer-4842620.xml")
root = tree.getroot()

# Extraer datos de la hoja "Tester Optimizator Results"
ns = {'ss': 'urn:schemas-microsoft-com:office:spreadsheet'}
rows = root.findall(".//ss:Worksheet[@ss:Name='Tester Optimizator Results']/ss:Table/ss:Row", ns)

# Leer encabezados
headers = [cell.find("ss:Data", ns).text for cell in rows[0]]

# Extraer datos
data = []
for row in rows[1:]:
    values = []
    for cell in row.findall("ss:Cell", ns):
        data_elem = cell.find("ss:Data", ns)
        values.append(data_elem.text if data_elem is not None else None)
    data.append(values)

# Crear DataFrame
df = pd.DataFrame(data, columns=headers)
df = df.dropna(how='all')  # eliminar filas completamente vacías
# Convertir columnas numéricas necesarias a tipo float
cols_to_convert = [
    "Sharpe Ratio", "Profit Factor", "Recovery Factor", "Equity DD %"
]
for col in cols_to_convert:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Calcular el score compuesto
df["Score"] = (
    df["Sharpe Ratio"] +
    df["Profit Factor"] +
    df["Recovery Factor"] -
    (df["Equity DD %"] / 10)
)

# Ordenar por score descendente
best_result = df.sort_values(by="Score", ascending=False).iloc[0]

best_result

Pass                         3047
Result                98.19447754
Profit               522.79773106
Expected Payoff          6.878918
Profit Factor            6.471056
Recovery Factor          8.342339
Sharpe Ratio            11.063829
Custom                          0
Equity DD %               14.1106
Trades                         76
max_orders                      4
orders_time_delay               8
max_spread                    188
stoploss                     3750
takeprofit                   1149
Score                   24.466164
Name: 3046, dtype: object